In [1]:
import numpy as np
from sklearn.utils import shuffle

def check_moment_conditions(model, batch_size=1, random_state=0):
    np.random.seed(random_state)

    # 1. Pick weights (start at initialization)
    w = model.initialize_weights()

    # 2. Full gradient
    full_grad = model.grad_F(w)
    grad_norm = np.linalg.norm(full_grad)

    # 3. Per-sample gradients
    per_sample_grads = np.array([model.grad_f_i(w, i) for i in range(model.m)])

    # 4. Mean stochastic gradient
    mean_grad = np.mean(per_sample_grads, axis=0)

    # --- First moment check ---
    bias_norm = np.linalg.norm(mean_grad - full_grad)
    rel_bias = bias_norm / (grad_norm + 1e-12)  # relative bias
    mu = (full_grad @ mean_grad) / (grad_norm ** 2 + 1e-12)  # alignment
    mu_G = np.linalg.norm(mean_grad) / (grad_norm + 1e-12)

    # --- Second moment check ---
    # Variance for each coordinate
    var_coords = np.var(per_sample_grads, axis=0, ddof=1)
    var_total = np.mean(np.sum((per_sample_grads - mean_grad)**2, axis=1))

    # M and M_V from Bottou's form
    M_V = max(0, (var_total - 1e-12) / (grad_norm**2 + 1e-12))
    M = var_total - M_V * grad_norm**2
    M_G = M_V + mu_G**2

    results = {
        "grad_norm": grad_norm,
        "rel_bias": rel_bias,
        "mu": mu,
        "mu_G": mu_G,
        "M": M,
        "M_V": M_V,
        "M_G": M_G
    }
    return results

# Example usage
# from your pipeline:
# X, y = load_healthcare_data()
# model = RegressionModel(X, y)
# moment_results = check_moment_conditions(model)
# print(moment_results)


In [2]:
from src.data.data_loaders import load_healthcare_data
from src.models.regression_model import RegressionModel

In [3]:
X, y = load_healthcare_data()
model = RegressionModel(X, y)
moment_results = check_moment_conditions(model)
print(moment_results)

{'grad_norm': 0.29217321392010853, 'rel_bias': 3.3320990695774386e-14, 'mu': 0.9999999999882867, 'mu_G': 0.9999999999965786, 'M': 2.3239632440663627e-11, 'M_V': 22.239351752570965, 'M_G': 23.239351752564122}


In [4]:
from src.utils.parameter_estimator import ParameterEstimator

estimator = ParameterEstimator(X, y, model, noise=0.01)
params = estimator.estimate_parameters()

print("Moment results:", moment_results)
print("Parameter estimator results:", {k: float(v) for k, v in params.items()})


Moment results: {'grad_norm': 0.29217321392010853, 'rel_bias': 3.3320990695774386e-14, 'mu': 0.9999999999882867, 'mu_G': 0.9999999999965786, 'M': 2.3239632440663627e-11, 'M_V': 22.239351752570965, 'M_G': 23.239351752564122}
Parameter estimator results: {'L': 1.647576769835411, 'c': 0.4609237567459406, 'mu': 0.9999999999882867, 'mu_G': 0.9999999999965786, 'M': 1.4050731901913376, 'M_V': 22.23935175258268, 'M_G': 23.23935175257584}
